<a href="https://colab.research.google.com/github/viniciusrpb/cloud_image_classification/blob/main/cloud_classification_ViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cloud Image Classification using Vision Transformers

In [16]:
#from google.colab import drive
#drive.mount('/content/drive')

In [17]:
#!cp -r "/content/drive/My Drive/img_satelite/classificacao/CCSN/train" "training"
#!cp -r "/content/drive/My Drive/img_satelite/classificacao/CCSN/val" "validation"
#!cp -r "/content/drive/My Drive/img_satelite/classificacao/CCSN/test" "testing"

In [18]:
#!pip install pytorch pytorch torchvision
#!pip install timm==0.3.2
#!pip install datasets transformers
#!pip install transformers pytorch-lightning --quiet
#!sudo apt -qq install git-lfs

In [19]:
from datasets import load_dataset
import tensorflow as tf
import torchvision
from torchvision.transforms import ToTensor
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import math
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, UnidentifiedImageError
from pathlib import Path
import torch
import glob
from huggingface_hub import HfApi, Repository
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchmetrics import Accuracy
from transformers import ViTFeatureExtractor,ViTForImageClassification,DeiTForImageClassification,BeitForImageClassification,DeiTFeatureExtractor,  BeitFeatureExtractor
from pytorch_lightning.callbacks import ModelCheckpoint

In [20]:
path_train = 'training'
path_validation = 'validation'
path_test = 'testing'

Define the image generator objects

In [21]:
train_ds = torchvision.datasets.ImageFolder(path_train, transform=ToTensor())
valid_ds = torchvision.datasets.ImageFolder(path_validation, transform=ToTensor())
test_ds = torchvision.datasets.ImageFolder(path_test, transform=ToTensor())

In [22]:
train_ds.classes

['Ac', 'As', 'Cb', 'Cc', 'Ci', 'Cs', 'Ct', 'Cu', 'Ns', 'Sc', 'St']

In [23]:
def fn_collator(batch):
    encodings = feature_extractor([x[0] for x in batch], return_tensors='pt')
    encodings['labels'] = torch.tensor([x[1] for x in batch], dtype=torch.long)
    return encodings 

Pega os códigos das classes do dataset

In [24]:
dic_label2id = {}
dic_id2label = {}
for i, class_name in enumerate(train_ds.classes):
  dic_label2id[class_name] = str(i)
  dic_id2label[str(i)] = class_name

Allocate objects for loading the data using the DataGenerator

In [25]:
from transformers import ViTFeatureExtractor

model_name_or_path = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)


loading feature extractor configuration file https://huggingface.co/google/vit-base-patch16-224-in21k/resolve/main/preprocessor_config.json from cache at /root/.cache/huggingface/transformers/7c7f3e780b30eeeacd3962294e5154788caa6d9aa555ed6d5c2f0d2c485eba18.c322cbf30b69973d5aae6c0866f5cba198b5fe51a2fe259d2a506827ec6274bc
Feature extractor ViTFeatureExtractor {
  "do_normalize": true,
  "do_resize": true,
  "feature_extractor_type": "ViTFeatureExtractor",
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "size": 224
}



In [26]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy","f1-score")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

In [27]:
from transformers import ViTForImageClassification

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(train_ds.classes),
    id2label=dic_id2label,
    label2id=dic_label2id)

loading feature extractor configuration file https://huggingface.co/google/vit-base-patch16-224-in21k/resolve/main/preprocessor_config.json from cache at /root/.cache/huggingface/transformers/7c7f3e780b30eeeacd3962294e5154788caa6d9aa555ed6d5c2f0d2c485eba18.c322cbf30b69973d5aae6c0866f5cba198b5fe51a2fe259d2a506827ec6274bc
Feature extractor ViTFeatureExtractor {
  "do_normalize": true,
  "do_resize": true,
  "feature_extractor_type": "ViTFeatureExtractor",
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "size": 224
}

loading configuration file https://huggingface.co/google/vit-base-patch16-224-in21k/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/7bba26dd36a6ff9f6a9b19436dec361727bea03ec70fbfa82b70628109163eaa.92995a56e2eabab0c686015c4ad8275b4f9cbd858ed228f6a08936f2c31667e7
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224-in21k",
  "architectures": [
    "ViTModel"
  ]

In [28]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vit-base-clouds",
  per_device_train_batch_size=2,
  evaluation_strategy="steps",
  num_train_epochs=10,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-5,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

PyTorch: setting up devices


In [29]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=fn_collator,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    tokenizer=feature_extractor,
)

Using amp half precision backend


In [30]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1774
  Num Epochs = 10
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 4440


Step,Training Loss,Validation Loss,Accuracy
100,2.076100,2.153402,0.308000
200,1.997900,1.971035,0.412000
300,1.850800,1.839117,0.452000
400,1.804900,1.800969,0.408000
500,1.444700,1.663477,0.512000
600,1.320200,1.619895,0.516000
700,1.305500,1.563865,0.520000
800,1.435800,1.551423,0.528000
900,1.247000,1.505095,0.540000
1000,1.346100,1.500200,0.544000


***** Running Evaluation *****
  Num examples = 250
  Batch size = 8
Saving model checkpoint to ./vit-base-clouds/checkpoint-100
Configuration saved in ./vit-base-clouds/checkpoint-100/config.json
Model weights saved in ./vit-base-clouds/checkpoint-100/pytorch_model.bin
Feature extractor saved in ./vit-base-clouds/checkpoint-100/preprocessor_config.json
Deleting older checkpoint [vit-base-clouds/checkpoint-400] due to args.save_total_limit
Deleting older checkpoint [vit-base-clouds/checkpoint-500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 250
  Batch size = 8
Saving model checkpoint to ./vit-base-clouds/checkpoint-200
Configuration saved in ./vit-base-clouds/checkpoint-200/config.json
Model weights saved in ./vit-base-clouds/checkpoint-200/pytorch_model.bin
Feature extractor saved in ./vit-base-clouds/checkpoint-200/preprocessor_config.json
Deleting older checkpoint [vit-base-clouds/checkpoint-600] due to args.save_total_limit
***** Running Evaluation

***** train metrics *****
  epoch                    =         10.0
  total_flos               = 1280399770GF
  train_loss               =       0.7832
  train_runtime            =   0:16:16.92
  train_samples_per_second =       18.159
  train_steps_per_second   =        4.545


In [31]:
metrics = trainer.evaluate(test_ds)
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** Running Evaluation *****
  Num examples = 519
  Batch size = 8


***** eval metrics *****
  epoch                   =       10.0
  eval_accuracy           =     0.5684
  eval_loss               =     1.3421
  eval_runtime            = 0:00:08.92
  eval_samples_per_second =     58.164
  eval_steps_per_second   =      7.284
